In [3]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import pickle
from keras.utils import to_categorical
import sys

epochs = 50
trainable=True

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:

sys.version_info[0]

3

In [3]:
def createMatrices(sentences, word2Idx, case2Idx, char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
        charIndices = []
#         labelIndices = []
        
        for word,char in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
            charIdx = []
            for x in char:
                charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
#             labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices,caseIndices, charIndices]) 
        
    return dataset


def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences

In [4]:
def doc_load(filename):
    f = open(filename,'rb')
    text = f.read().decode()
    f.close()
    return text

def save_doc(lines, filename):
    data = '\n'.join(lines)
    f = open(filename,'wb')
    f.write(data.encode())
    f.close()


In [5]:
text = doc_load('/home/jindal/notebooks/wikipedia/de-wiki_tokenized.txt')

In [6]:
print(text[:200])

Alan Smithee

Alan Smithee steht als Pseudonym für einen fiktiven Regisseur , der Filme verantwortet , bei denen der eigentliche Regisseur seinen Namen nicht mit dem Werk in Verbindung gebracht haben 


In [7]:
import string
def clean_doc(doc):
#     doc = doc.replace('--',' ')
    tokens = doc.split()
    print('done split')
    table = str.maketrans('','', string.punctuation) # remove the punctuation 
    tokens = [w.translate(table) for w in tokens]
    print('done1')
    tokens = [word for word in  tokens if word.isalpha()] # only take words. No numbers etc
    print('done2')
#     tokens = [word.lower() for word in tokens]
#     print('done3')
    return tokens

In [8]:
tokens = clean_doc(text)

done split
done1
done2


In [9]:
print(len(tokens))

713941425


In [ ]:
# tokens = tokens[:50000]

In [10]:
import pickle

with open('tokens.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(tokens, f)

In [5]:
tokens = pickle.load(open('tokens.pkl','rb'))

In [6]:
print(len(tokens))
print(len(set(tokens)))

713941425
7237133


In [13]:
print(tokens[0])

Alan


In [ ]:
length = 50+1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
    
print(len(sequences))

In [ ]:
print(sequences[0])

In [ ]:
save_doc(sequences, 'wikipedia_clean.txt')

In [ ]:
doc = doc_load('/home/jindal/notebooks/jindal/NER/wikipedia_clean.txt')
lines = doc.split('\n')

In [ ]:
print(lines[:2])

In [ ]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [ ]:
sentences=[]
for i,line in enumerate(lines):
    splits = line.split()
    sentence = []
    for word in splits:
#         print(word)
        temp = []
        char = []
        for c in word:
            char.append(c)
        temp.append(word)
        temp.append(char)
        sentence.append(temp)
    sentences.append(sentence)

In [ ]:
print(sentences[0])

In [ ]:
with open('pickle.txt','wb') as f:
    pickle.dump(sentences,f)

In [ ]:
sentences = pickle.load(open('pickle.txt','rb'))

In [ ]:
words = {}
characters={}

for sentence in sentences:
    for word in sentence:
        token = word[0]
        words[token.lower()]=True
        for char in word[1]:
            characters[char]=True

In [ ]:
print(characters)

In [ ]:
with open('/home/jindal/notebooks/fastText/wiki.de.txt','wb') as f:
    all_words = ' '.join(words.keys())
    f.write(all_words.encode())

In [ ]:
print(len(set(words.keys())))

In [ ]:
word2Idx={}
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('wikipedia_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [ ]:
print(len(wordEmbeddings[3]))

In [ ]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

In [ ]:
print(len(sentences))

In [ ]:
trainSentences = sentences[:40000]
testSentences = sentences[40000:45000]
devSentences = sentences[45000:]

In [ ]:
print(trainSentences[0])

In [ ]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx, case2Idx, char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx , case2Idx, char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx,case2Idx, char2Idx))

In [ ]:
# first is the index for word. 2nd for casing. 3rd for the char
print(train_set[0])
print(len(train_set[0][0]))
print(len(train_set[0][1]))


In [ ]:
word_inp=[]
casing_inp=[]
char_inp = []

In [ ]:
with open("train_set_wikipedia.txt",'wb') as f:
    pickle.dump(train_set,f)

In [ ]:
y=[]
for sentence in train_set:
    word = sentence[0][50]
    y.append(word)
    del sentence[0][-1]
    sentence[2] = sentence[2][:-1]
    del sentence[1][1]
    word_inp.append(sentence[0])
    casing_inp.append(sentence[1])
    char_inp.append(sentence[2])

In [ ]:
print(len(word_inp[0]))
print(len(casing_inp[0]))
print(len(char_inp[0]))

In [ ]:
vocab_size = len(word2Idx.keys())+1
print(vocab_size)

In [ ]:
print(y[:10])

In [ ]:
# y = to_categorical(y, num_classes = vocab_size)

In [ ]:
# print(len(y[0]))

In [ ]:
words_input = Input(shape=(None,), dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words,char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.25))(output)
# output = TimeDistributed(Dense(vocab_size))(output)
# crf = CRF(len(label2Idx))
# output = crf(output)
# output = Flatten()(output)
output = Dense(vocab_size, activation='softmax')(output)
# output = Flatten()(output)
# output = Dense(vocab_size, activation='softmax')(output)
# model.add(Dense(vocab_size, activation='softmax'))
model = Model(inputs=[words_input,casing_input, character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
plot_model(model, to_file='model.png')

In [ ]:
y = np.array(y)
word_inp = np.array(word_inp)
casing_inp = np.array(casing_inp)
char_inp = np.array(char_inp)

In [ ]:
print(word_inp.shape)
print(casing_inp.shape)
print(char_inp.shape)

In [ ]:
model.fit([word_inp,casing_inp, char_inp],y,epochs=1)

In [ ]:
model.save('german_ner_lm.h5')